# Нейронные сети. Понятно. Детально.

## Раздел 7. Разработка фреймворка глубокого обучения.

<h1>Разработка фреймворка глубокого обучения</h1>

В качестве иллюстрации всех описанных идей разработаем фреймворк машинного обучения подобный PyTorch.

<h2> Архитектура </h2>

В данном проекте будет 4 основных вида объектов:
* тензор - основной класс фреймворка, отвечает за всю основную обработку
* операции - классы отвечающие за конкреиные реализации операций
* функции - функции, которые дают интерфейс для доступа к некоторым операциям в виде функций, а не методов класса
* модели - слои нейронных сетей и другие модели необходимые для фреймворка

Данный фремворк написан в учебных целях и как демонстрация математических принципов лежащих в основе работы нейронных сетей, поэтому в нем используется малое уоличество приемов оптимизации и численных алгоритмов.

<h2>Тензор</h2>

Начнем рассмотрение данного фреймворка с онсновного класса тензора.

Для корректной работы необходимо подкоючить следующие заголовочные файлы:

In [1]:
import numpy as np

Конструктор класса имеет следующий вид:

In [2]:
class Tensor:

    def __init__(self, data, calc_grad=False):
        self._data = np.array(data)

        self._old_shape = None
        if self._data.ndim == 0:
            self._old_shape = self._data.shape
            self._data = self._data[None, None]
        elif self._data.ndim == 1:
            self._old_shape = self._data.shape
            self._data = self._data[..., None]

        self._calc_grad = calc_grad
        self._operation = operations.VariableOperation(self)
        self._grad = None

Класс Tensor содержит слелующие основные поля:
* _data - данные хранимые в виде numpy array
* _calc_grad - переменная, показывающая необходимость расчета градиента
* _operation - операция, пораждающая данный тензор
* _grad - градиент рассчитанный для данного тензора (если такой расчет производился)

In [3]:
class Tensor:


    @property
    def data(self):
        if self._old_shape is not None:
            return self._data.reshape(self._old_shape)
        else:
            return self._data

    
    @property
    def matrix(self):
        return self._data

    
    @property
    def calc_grad(self):
        return self._calc_grad

    
    @property
    def grad(self):
        if self._grad is not None:
            if self._old_shape is not None:
                return Tensor(self._grad._data.reshape(self._old_shape), calc_grad=self._calc_grad)
            else:
                return self._grad
        else:
            return None


    @property
    def grad_matrix(self):
        return self._grad


    @grad_matrix.setter
    def grad_matrix(self, grad):
        self._grad = grad


    @property
    def T(self):
        return self.t()

    
    def __repr__(self):
        return str(self.data)

Методы data и matrix дают доступ к данным, но метод matrix возвращает данные всегда приведенные в наиболее удобную для расчетов форму. Метод data всегда возвращает данные в том виде, в котором они были заданны.

Данная проблема вызвана тем, что для расчетов вектора должны быть в форме 
$\vec v = \begin{pmatrix} 
     x_1 \\
     x_2 \\
     ... \\
     x_n
\end{pmatrix}$, а массивы зачастую храняться и задаются в виде $\vec v = (x_1, x_2, ... c_n)$.

In [4]:
class Tensor:  

    def forward(self):
        return self._operation.forward()

    def backward(self, tensor=None):
        if self.calc_grad:
            self._operation.backward(tensor)

Операция forward вечает за вычисление операции порадившей тензор. Операция backward отвечает за расчет обратного хода алгоритма градиентного спуска.

Для упрощения кода и объединения всех однотипных действий используется функция __prepare_operation.

In [5]:
class Tensor:
    
    def __prepare_operation(self, operation, other_grad=False):
        data = operation.forward()
        data._operation = operation
        return data

Далее идут унарные операции.

In [6]:
class Tensor:
    
    def sqr(self):
        op = operations.SqrOperation(self)
        return self.__prepare_operation(op)

    def sqrt(self):
        op = operations.SqrtOperation(self)
        return self.__prepare_operation(op)

    def exp(self):
        op = operations.ExpOperation(self)
        return self.__prepare_operation(op)

    def sigmoid(self):
        op = operations.SigmoidOperation(self)
        return self.__prepare_operation(op)


Операция транспонирования.

In [7]:
class Tensor:
    
    def t(self):
        op = operations.TransposeOperation(self)
        return self.__prepare_operation(op)

Операция суммирования.

In [8]:
 class Tensor:
    
    def sum(self):
        op = operations.SumOperation(self)
        return self.__prepare_operation(op)

Операции hardmax и softmax.

In [9]:
class Tensor:
    
    def softmax(self):
        op = operations.SoftMaxOperation(self)
        return self.__prepare_operation(op)

    def hardmax(self):
        op = operations.HardMaxOperation(self)
        return self.__prepare_operation(op)

Операция индексирования.

In [10]:
class Tensor:
    
    def __getitem__(self, item):
        op = operations.IndexOperation(self, item)
        return self.__prepare_operation(op)

Бинарные операции поиска минимума и максимума.

In [11]:
class Tensor:
    
    def min(self, other):
        op = operations.MinOperation(self, other)
        return self.__prepare_operation(op)


    def max(self, other):
        op = operations.MaxOperation(self, other)
        return self.__prepare_operation(op)


Бинарные операции: +, -, /, *.

In [12]:
class Tensor:

    def __add__(self, other):
        op = operations.AddOperation(self, other)
        return self.__prepare_operation(op)


    def __sub__(self, other):
        op = operations.SubOperation(self, other)
        return self.__prepare_operation(op)


    def __mul__(self, other):
        op = operations.MulOperation(self, other)
        return self.__prepare_operation(op)


    def __truediv__(self, other):
        op = operations.DivOperation(self, other)
        return self.__prepare_operation(op)   

Операция матричного умножения.

In [13]:
 def __matmul__(self, other):
        op = operations.MatMulOperation(self, other)
        return self.__prepare_operation(op)


<h2> Утилитарные функции </h2>

В данном разделе представленны все утилитарные функции, использующиеся в фреймворке.

Для вычислений потребуются функции сравнения размеров двух тензоров. Функции shape_greater и shape_less сравнивает два размера тензоров.

In [14]:
def shape_greater(shape1, shape2):
    if len(shape1) == len(shape2):
        if shape1 > shape2:
            return True
        else:
            return False
    elif len(shape1) > len(shape2):
        return True
    else:
        return False

def shape_less(shape1, shape2):
    if len(shape1) == len(shape2):
        if shape1 < shape2:
            return True
        else:
            return False
    elif len(shape1) < len(shape2):
        return True
    else:
        return False

Транспонирование матрицы, тензора и многомерной матрицы отличаются в значительной степени. Основной операцие фреймворка является транспонирование многомерной матрицы. Оно отличается от транспонирования вектора тем, что меняются местами только два последних измерения.

In [15]:
def grad_transpose(np_tensor):
    if np_tensor.ndim == 0:
        return np.array([[np_tensor]])
    elif np_tensor.ndim == 1:
        return np.expand_dims(np_tensor)
    elif np_tensor.ndim == 2:
        return np_tensor.T
    else:
        return np.swapaxes(np_tensor, -1, -2)

Далее идет операция уменьшения размерности многомерной матрицы за счет суммирования по координатам.

In [16]:
def grad_zip_variable(np_tensor, np_same_as):
    shape_in = np_tensor.shape
    shape_out = np_same_as.shape

    if shape_in == shape_out:
        return np_tensor
    else:
        res = []
        for i, (si, so) in enumerate(itertools.zip_longest(shape_in[::-1], shape_out[::-1])):
            if si != so:
                res.append(-i - 1)
        return np.sum(np_tensor, axis=tuple(res)).reshape(np_same_as.shape)

Операция увеличения размерности матрицы за счет операции broadcasting.

In [17]:
def grad_extend(np_tensor, np_same_as):
    return np.broadcast_arrays(np_tensor, np_same_as)[0]

Нормализация размера матрицы для выполнения бинарных операций при обратном течении градиента.

In [18]:
def grad_reshape(np_tensor, np_same_as):
    shape1 = np_tensor.shape
    shape2 = np_same_as.shape
    if shape1 == shape2:
        return np_tensor
    elif shape_greater(shape1, shape2):
        return grad_zip_variable(np_tensor, np_same_as)
    else:
        return grad_extend(np_tensor, np_same_as)

<h2>Операции над тензорами</h2>

В данном разделе будут приведены коды непосредственного дифференциирования различных операций.

Корневой класс операции.

In [19]:
class Operation:

    def __init__(self):
        self._res = None

    def forward(self):
        pass

    def backward(self, np_tensor):
        pass

Класс для обработки переменных.

In [20]:
class VariableOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        self._res = Tensor(self._x.matrix, self._x.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            if self._x.grad_matrix is None:
                if np_tensor is None:
                    self._x.grad_matrix = Tensor(np.ones_like(self._x.matrix))
                else:
                    self._x.grad_matrix = Tensor(grad_reshape(np_tensor, self._x.matrix))
            else:
                if np_tensor is None:
                    self._x.grad_matrix = Tensor(grad_reshape(np.ones_like(self._x.matrix) + self._x.matrix, self._x.matrix))
                else:
                    self._x.grad_matrix = Tensor(grad_reshape(self._x.grad_matrix.matrix + np_tensor, self._x.matrix))

Класс для обработки транспонирования.

In [21]:
class TransposeOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        self._res = Tensor(self._x.matrix.T, self._x.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            if np_tensor is None:
                self._x.backward(np.ones_like(self._x.matrix))
            else:
                self._x.backward(np_tensor.T)

Класс для обработки операции суммирования тензора.

In [22]:
class SumOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        self._res = Tensor((np.sum(self._x.matrix),), self._x.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            tensor = np.ones_like(self._x.matrix)
            if np_tensor is None:
                self._x.backward(grad_reshape(np_tensor, self._x.matrix))
            else:
                self._x.backward(grad_reshape(np_tensor, self._x.matrix))

Класс для обработки операции softmax.

In [23]:
class SoftMaxOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        exp = self._x.exp()
        sum = exp.sum()
        self._res = exp / sum
        return Tensor(self._res.matrix, self._res.calc_grad)

    def backward(self, np_tensor):
        if self._x.calc_grad:
            self._res.backward(np_tensor)

Класс для обработки операции hardmax.

In [24]:
class HardMaxOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        sum = self._x.sum()
        self._res = self._x / sum
        return self._x / sum

    def backward(self, np_tensor):
        if self._x.calc_grad:
            self._res.backward(np_tensor)

Класс для обработки операции возведения в квадрат.

In [25]:
class SqrOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        self._res = Tensor(self._x.matrix * self._x.matrix, self._x.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            data = 2 * self._x.matrix
            if np_tensor is None:
                self._x.backward(data)
            else:
                self._x.backward(np_tensor * data)

Класс для извлечения квадратного корня.

In [26]:
class SqrtOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        self._res = Tensor(np.sqrt(self._x.matrix), self._x.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            data = 0.5 / np.sqrt(self._x.matrix)
            if np_tensor is None:
                self._x.backward(data)
            else:
                self._x.backward(np_tensor * data)

Класс для вычисления экспоненты.

In [27]:
class ExpOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        self._res = Tensor(np.exp(self._x.matrix), self._x.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            data = np.exp(self._x.matrix)
            if np_tensor is None:
                self._x.backward(data)
            else:
                self._x.backward(np_tensor * data)

Класс для обработки сигмоиды.

In [28]:
class SigmoidOperation(Operation):

    def __init__(self, x):
        super().__init__()
        self._x = x

    def forward(self):
        ex = np.exp(self._x.matrix)
        self._res = Tensor(ex / (ex + 1), self._x.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            data = self._res.matrix * (1 - self._res.matrix)
            if np_tensor is None:
                self._x.backward(data)
            else:
                self._x.backward(np_tensor * data)


Обработка операции поиска минимума двух многомерных матриц.

In [29]:
class MinOperation(Operation):
    def __init__(self, x, y):
        super().__init__()
        self._x = x
        self._y = y

    def forward(self):
        self._res = tensor.Tensor(np.minimum(self._x.matrix, self._y.matrix), self._x.calc_grad or self._y.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            if np_tensor is None:
                one = np.ones_like(self._res.matrix)
                res = np.zeros_like(self._res.matrix)
                idx = self._x.matrix <= self._y.matrix
                res[idx] = one[idx]
                self._x.backward(grad_reshape(res, self._x.matrix))
            else:
                res = np.zeros_like(np_tensor)
                idx = self._x.matrix <= self._y.matrix
                res[idx] = np_tensor[idx]
                self._x.backward(grad_reshape(res, self._x.matrix))
        if self._y.calc_grad:
            if np_tensor is None:
                one = np.ones_like(self._res.matrix)
                res = np.zeros_like(self._res.matrix)
                idx = self._x.matrix > self._y.matrix
                res[idx] = one[idx]
                self._y.backward(grad_reshape(res, self._y.matrix))
            else:
                res = np.zeros_like(np_tensor)
                idx = self._x.matrix > self._y.matrix
                res[idx] = np_tensor[idx]
                self._y.backward(grad_reshape(res, self._y.matrix))

Обработка операции поиска максимума двух многомерных матриц.

In [30]:
class MaxOperation(Operation):
    def __init__(self, x, y):
        super().__init__()
        self._x = x
        self._y = y

    def forward(self):
        self._res = Tensor(np.maximum(self._x.matrix, self._y.matrix), self._x.calc_grad or self._y.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            if np_tensor is None:
                one = np.ones_like(self._res.matrix)
                res = np.zeros_like(self._res.matrix)
                idx = self._x.matrix >= self._y.matrix
                res[idx] = one[idx]
                self._x.backward(grad_reshape(res, self._x.matrix))
            else:
                res = np.zeros_like(np_tensor)
                idx = self._x.matrix >= self._y.matrix
                res[idx] = np_tensor[idx]
                self._x.backward(grad_reshape(res, self._x.matrix))
        if self._y.calc_grad:
            if np_tensor is None:
                one = np.ones_like(self._res.matrix)
                res = np.zeros_like(self._res.matrix)
                idx = self._x.matrix < self._y.matrix
                res[idx] = one[idx]
                self._y.backward(grad_reshape(res, self._y.matrix))
            else:
                res = np.zeros_like(np_tensor)
                idx = self._x.matrix < self._y.matrix
                res[idx] = np_tensor[idx]
                self._y.backward(grad_reshape(res, self._y.matrix))



Обработка операции сложения двух многомерных матриц.

In [31]:
class AddOperation(Operation):
    def __init__(self, x, y):
        super().__init__()
        self._x = x
        self._y = y

    def forward(self):
        self._res = Tensor(self._x.matrix + self._y.matrix, self._x.calc_grad or self._y.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            if np_tensor is None:
                self._x.backward(np.ones_like(self._res.matrix))
            else:
                self._x.backward(grad_reshape(np_tensor, self._x.matrix))
        if self._y.calc_grad:
            if np_tensor is None:
                self._y.backward(np.ones_like(self._res.matrix))
            else:
                self._y.backward(grad_reshape(np_tensor, self._y.matrix))

Обработка операции поиска разности двух многомерных матриц.

In [32]:
class SubOperation(Operation):
    def __init__(self, x, y):
        super().__init__()
        self._x = x
        self._y = y

    def forward(self):
        self._res = Tensor(self._x.matrix - self._y.matrix, self._x.calc_grad or self._y.calc_grad)
        return self._res


    def backward(self, np_tensor):
        if self._x.calc_grad:
            if np_tensor is None:
                self._x.backward(np.ones_like(self._res.matrix))
            else:
                self._x.backward(grad_reshape(np_tensor, self._x.matrix))
        if self._y.calc_grad:
            if np_tensor is None:
                self._y.backward(- np.ones_like(self._res.matrix))
            else:
                self._y.backward(grad_reshape(- np_tensor, self._y.matrix))


Обработка операции произведения двух многомерных матриц.

In [33]:
class MulOperation(Operation):
    def __init__(self, x, y):
        super().__init__()
        self._x = x
        self._y = y

    def forward(self):
        self._res = Tensor(self._x.matrix * self._y.matrix, self._x.calc_grad or self._y.calc_grad)
        return self._res


    def backward(self, np_tensor):
        if self._x.calc_grad:
            if np_tensor is None:
                self._x.backward(grad_reshape(self._y.matrix, self._x.matrix))
            else:
                self._x.backward(grad_reshape(np_tensor * self._y.matrix, self._x.matrix))
        if self._y.calc_grad:
            if np_tensor is None:
                self._y.backward(grad_reshape(self._x.matrix, self._y.matrix))
            else:
                self._y.backward(grad_reshape(np_tensor * self._x.matrix, self._y.matrix))

Обработка операции деления двух многомерных матриц.

In [34]:
class DivOperation(Operation):
    def __init__(self, x, y):
        super().__init__()
        self._x = x
        self._y = y

    def forward(self):
        self._res = Tensor(self._x.matrix / self._y.matrix, self._x.calc_grad or self._y.calc_grad)
        return self._res


    def backward(self, np_tensor):
        if self._x.calc_grad:
            if np_tensor is None:
                self._x.backward(grad_reshape(1 / self._y.matrix, self._x.matrix))
            else:
                self._x.backward(grad_reshape(np_tensor / self._y.matrix, self._x.matrix))
        if self._y.calc_grad:
            if np_tensor is None:
                self._y.backward(grad_reshape(- self._x.matrix / (self._y.matrix * self._y.matrix), self._y.matrix))
            else:
                self._y.backward(grad_reshape(np_tensor * (-self._x.matrix / (self._y.matrix * self._y.matrix)), self._y.matrix))

Обработка операции матричного умножения двух многомерных матриц.

In [35]:
class MatMulOperation(Operation):

    def __init__(self, x, y):
        super().__init__()
        self._x = x
        self._y = y

    def forward(self):
        self._res = Tensor(self._x.matrix @ self._y.matrix, self._x.calc_grad or self._y.calc_grad)
        return self._res

    def backward(self, np_tensor):
        if self._x.calc_grad:
            if np_tensor is None:
                self._x.backward(grad_reshape(grad_transpose(self._y.matrix), self._x.matrix))
            else:
                self._x.backward(grad_reshape(np_tensor @ grad_transpose(self._y.matrix), self._x.matrix))
        if self._y.calc_grad:
            if np_tensor is None:
                self._y.backward(grad_reshape(grad_transpose(self._x.matrix), self._y.matrix))
            else:
                self._y.backward(grad_reshape(grad_transpose(self._x.matrix) @ np_tensor, self._y.matrix))


Обработка операции индексирования двух многомерных матриц.

In [36]:
class IndexOperation(Operation):
    def __init__(self, x, index):
        super().__init__()
        self._x = x
        self._index = index

    def forward(self):
        zero = np.zeros_like(self._x.matrix)
        data = self._x.matrix[self._index]
        zero[self._index] = data
        self._res = Tensor(zero)
        return Tensor(data, self._x.calc_grad)

    def backward(self, np_tensor):
        if self._x.calc_grad:
            # ToDo check
            if np_tensor is None:
                ex_tensor = np.zeros_like(self._x.matrix)
                ex_tensor[self._index] = np.ones_like(self._res.matrix)[self._index]
                self._x.backward(ex_tensor)
            else:
                ex_tensor = np.zeros_like(self._x.matrix)
                #!!!
                #ex_tensor[self._index] = np_tensor.squeeze()
                # ToDo ???
                ex_tensor[self._index] = np_tensor #.squeeze()
                self._x.backward(ex_tensor)




<h2> Функции </h2>

В данном разделе приведены интерфейсные функции для более удобных расчетов. Все приведенные функции полностью зеркалируют соответствующие функции, приведенные в разделе с описанием класса тензора.

In [37]:
def min(x, y):
    return x.min(y)


def max(x, y):
    return y.max(y)

def transpose(x):
    return x.T

def sum(x):
    return x.sum()

def hardmax(x):
    return x.hardmax()

def softmax(x):
    return x.softmax()

def sqr(x):
    return x.sqr()

def sqrt(x):
    return x.sqrt()

def exp(x):
    return x.exp()

def sigmoid(x):
    return x.sigmoid()

def add(x, y):
    return x + y

def sub(x, y):
    return x - y

def div(x, y):
    return x / y

def mul(x, y):
    x * y

def matmul(x, y):
    return x * y



<h2> Слои нейронной сети </h2>

Разработаем на основе приведенного фреймворка нейронную сеть. Для этого необходимо создать корневой класс, от которого будут наследоваться все остальные слои сети.

In [38]:
class Model:
    def __init__(self):
        self._parameters = []

    def __call__(self, *args, **kwargs):
        return self.forward(args[0])

    def forward(self, x):
        return x

    @property
    def parameters(self):
        return self._parameters

    def zero_grad(self):
        for i in self._parameters:
            if i.grad is not None:
                i.grad.matrix.fill(0)


Разработаем на его основе линейный слой нейронной сети.

In [39]:
class Linear(Model):

    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self._in_features = in_features
        self._out_features = out_features
        self._bias = bias

        data = np.random.random((in_features, out_features))

        self._w = Tensor(data, True)
        self._parameters.append(self._w)

        if self._bias:
            self._b = Tensor([1.], True)
            self._parameters.append(self._b)



    def forward(self, x):
        if self._bias:
            return x @ self._w + self._b
        else:
            return x @ self._w


<h2> Пример обучения нейронной сети на данном фреймворке </h2>

Рассмотрим конкретный пример обучения нейронной сети на разработанном фреймворке.

In [40]:
from tensor import Tensor
from model import Linear

alpha = 0.001
epochs = 1000

x = [[1., 2], [2., 3.], [3., 4.], [4., 5.], [5., 6.]]
y = [1., 2., 3., 4., 5.]

x = Tensor(x)
y = Tensor(y)

m = Linear(2, 1)

for i in range(epochs):
    m.zero_grad()
    z = (m(x) - y).sqr().sum()
    z.backward()
    for j in m.parameters:
        j.update(j.matrix - alpha * j.grad.matrix)
    if i % 100 == 0:
        print('epoch: [{}] loss: [{}]'.format(i, z))

p = m.parameters
w = p[0]
b = p[1]

print('w=', w.data)
print('b=', b.data)


epoch: [0] loss: [[8.76790461]]
epoch: [100] loss: [[0.06511864]]
epoch: [200] loss: [[0.00122022]]
epoch: [300] loss: [[1.61212188e-05]]
epoch: [400] loss: [[6.34445084e-07]]
epoch: [500] loss: [[6.13473101e-08]]
epoch: [600] loss: [[4.0658064e-09]]
epoch: [700] loss: [[1.89654001e-10]]
epoch: [800] loss: [[6.59559012e-12]]
epoch: [900] loss: [[1.69769416e-13]]
w= [[0.2760121 ]
 [0.72398792]]
b= [-0.72398796]
